In [1]:
"""ATE: Automatic Term Extraction
"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import MeCabTokenizer

In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [3]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [6]:
print("\n".join(docs[2023][:15]))

生体医工学ウェブ辞典（第二分冊）
医療機関等への建築ガイドライン浸透のための課題と今後の展望
メーカーから見た医用テレメータ設置・運用に関する現状と確認事項
医用テレメータの概要と医療現場での現状と希望
医療機関における電波利用機器に配慮した建築ガイドライン・同解説－医用テレメータ編－のポイント
生体医工学×医療者教育―効率的な知識・技術発展のためにー
生体医工学×知のデザイン －臨床研究における学際の必要性－
生体医工学×医工連携～自身の開発経験談～
生体医工学×医療機器管理のエビデンス創設を目指して
スマートコントラクトを用いたトラストデータ流通システム
聴診音解析システムを活用した医療DXへの挑戦
看護理工学的思考によるリアルワールドデータの活用
皮膚感覚メカニズム解明のためのセンシング・ディスプレイ技術
神経オルガノイドを用いた特殊培養デバイスによる疼痛評価システム
次世代網膜刺激型人工視覚デバイスの開発


In [7]:
year = 2019

tok = MeCabTokenizer() 
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題',"および", "評価"]
ignores1 = ['型']
ignores2 = ['中','的','時','システム','用']

compound_nouns = []
for y in years:
    for sentence in docs[y]:
        if sentence.isascii():
            continue 
    
        tokens = tok.iter_token(sentence)
        nouns = tok.extract_nouns(tokens)
        # print(nouns)
        nouns = tok.filter_nouns(nouns, n=1, stopwords=ignores, stopwords_begin=ignores1, stopwords_end=ignores2)
        # print(nouns)
        if len(nouns) > 0 :
            compound_nouns.extend(nouns)

lr = LRValue()
# lr_values = lr.fit_transform(compound_nouns)
lr_values = lr.fit(compound_nouns).transform(compound_nouns, mode=3)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)
list(data_collection)[:50]

/tmp/working/notebooks/theme_river/termextract/termextract.py:53: RuntimeWarning: invalid value encountered in power
  return np.power(a.cumprod()[-1], 1/(2 * len(compound_noun)))


[('性', 104.69479452198185),
 ('計測', 79.87490219086344),
 ('画像 計測', 71.09330215402213),
 ('機能 計測', 69.24706411841348),
 ('細胞', 67.6904720030818),
 ('細胞 機能', 63.74703163941054),
 ('画像', 63.2771680782255),
 ('細胞 運動', 63.15513972885417),
 ('計測 解析', 61.48471641170195),
 ('細胞 モデル', 61.25183078345497),
 ('計測 光', 60.61555494078824),
 ('光 計測', 60.61555494078824),
 ('機能', 60.03332407921454),
 ('脳 機能 計測', 59.93529742122243),
 ('生体 画像 計測', 59.754471259142555),
 ('運動 機能', 59.475913985403544),
 ('超音波 画像 計測', 59.44879840197806),
 ('運動', 58.92367945062494),
 ('生体 機能 計測', 58.715422927581805),
 ('生体 計測', 58.06736174869173),
 ('画像 刺激', 57.19700083997363),
 ('刺激 画像', 57.19700083997363),
 ('細胞 組織', 56.689450737053555),
 ('細胞 解析', 56.60121786151975),
 ('光 脳 機能 計測', 56.098526812357065),
 ('モデル', 55.42562584220407),
 ('血管 細胞', 53.99452807653034),
 ('呼吸 機能 計測', 53.76080874400938),
 ('自動 計測', 53.56148350403636),
 ('細胞 センサ', 52.00223301501167),
 ('脳 機能', 51.918104413807754),
 ('刺激', 51.70106381884226),
 ('生体 画像'

In [38]:
terms = lr_values
freq = lr.compound_noun

terms.sort(key=lambda z: (len(z[0].split()), len(z[0]),-z[1]), reverse=False)
n, m = 1800, 1
for i, v in terms[n:]:
    word = "".join(i.split())
    if len(word) >= m and freq[i]>2:
        print(f"{word} & ${v:.3f}$ & ${freq[i]}$\\\\ ")

AMED & $1.000$ & $3$\\ 
事業計画 & $1.000$ & $4$\\ 
脳脊髄液 & $1.000$ & $4$\\ 
パラメータ & $12.961$ & $5$\\ 
メカニズム & $9.381$ & $3$\\ 
センシング & $8.485$ & $3$\\ 
fNIRS & $7.416$ & $14$\\ 
モデリング & $5.099$ & $5$\\ 
タンパク質 & $4.899$ & $4$\\ 
ヘルスケア & $4.472$ & $6$\\ 
数理モデル & $4.000$ & $16$\\ 
マイクロ波 & $3.742$ & $3$\\ 
自律神経系 & $3.162$ & $5$\\ 
プログラム & $3.000$ & $10$\\ 
周波数特性 & $2.449$ & $3$\\ 
有限要素法 & $2.000$ & $3$\\ 
アプローチ & $1.732$ & $5$\\ 
メラノーマ & $1.732$ & $4$\\ 
SSVEP & $1.414$ & $4$\\ 
認知症予防 & $1.414$ & $8$\\ 
組み合わせ & $1.414$ & $3$\\ 
乳がん検診 & $1.414$ & $3$\\ 
個別化医療 & $1.000$ & $3$\\ 
主成分分析 & $1.000$ & $3$\\ 
アカデミア & $1.000$ & $3$\\ 
酸化チタン & $1.000$ & $7$\\ 
機械的性質 & $1.000$ & $4$\\ 
アスリート & $1.000$ & $3$\\ 
モニタリング & $15.875$ & $6$\\ 
アルゴリズム & $8.832$ & $3$\\ 
ウェアラブル & $8.307$ & $3$\\ 
ダイナミクス & $4.472$ & $4$\\ 
視覚誘発電位 & $4.000$ & $6$\\ 
フレキシブル & $3.464$ & $3$\\ 
ビッグデータ & $3.464$ & $3$\\ 
加速度センサ & $2.449$ & $9$\\ 
独立成分分析 & $2.449$ & $3$\\ 
間葉系幹細胞 & $2.449$ & $5$\\ 
臨床工学技士 & $2.449$ & $6$\\ 
3Dプリンタ & $2.

In [ ]:
n = 3
